In [ ]:
# Imports
import pandas as pd
import numpy as np
import duckdb

In [ ]:
# DuckDB connection
conn = duckdb.connect()

In [ ]:
# Load dnc source files
df1 = conn.execute("""
    SELECT *
    FROM read_csv_auto('dataset/*.csv')
""").df()

df2 = conn.execute("""
    SELECT *
    FROM read_csv_auto('dataset/DNC5/DNC5.csv')
""").df()

In [ ]:
# Convert multiple columns into single column
melted_df1 = df1.melt()
melted_df2 = df2.melt()

dropped_column1 = melted_df1.drop(['variable'], axis=1)
dropped_column2 = melted_df2.drop(['variable'], axis=1)

In [ ]:
# Merge two dataframes/DNC
dnc = pd.concat([dropped_column1, dropped_column2], ignore_index=True)

In [ ]:
# Update data type to int
dnc['value'] = dnc['value'].fillna(0)
dnc['value'] = dnc['value'].astype(np.int64)

In [ ]:
# Export to csv
conn.execute("""
    COPY dnc TO 'dnc_list_final.csv' (DELIMITER ',')
""").df()